<a href="https://colab.research.google.com/github/qsardor/GoogleColabProjects/blob/main/Colab_NO_NVC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🖥️ Colab RDP with noVNC (Browser Access)

This notebook sets up a full remote desktop (XFCE) on your Colab instance and makes it accessible **directly in your browser** using noVNC and an ngrok tunnel. No client is needed.

## ⚠️ IMPORTANT: Temporary Storage
> This Colab instance is temporary. **Any files you save *inside* the RDP (on the Desktop, in Documents, etc.) will be DELETED when the machine disconnects.**
>
> To save your work permanently, check the **'mount_google_drive'** box before running. This will add your Google Drive folder to the RDP's file manager.

## 📝 Instructions
---
1.  **Add your ngrok Token:**
    * Click the **Key icon (🔑)** in the left sidebar to open **Secrets**.
    * Create a new secret with the name `NGROK`.
    * Paste your ngrok authtoken (from https://dashboard.ngrok.com/get-started/your-authtoken) into the **Value** field.
    * Make sure **'Notebook access'** is enabled for this secret.
2.  **Password is `root`:** The VNC password for this session is fixed to `root` and cannot be changed.
3.  **Run the Cell:** Press the "Play" button.
4.  **Wait:** The setup will take 5-10 minutes.
5.  **Connect:** Click the public `.ngrok.io` link that appears. When prompted, enter the VNC password (`root`) to connect.

## Credits & Resources
---
* **Colab by:** Q.SARDOR ([Telegram](https://t.me/qsardorblog) | [GitHub](https://github.com/qsardor/))
* **More Colab Projects:** [Explore on GitHub](https://github.com/qsardor/GoogleColabProjects)


In [ ]:
#@title ▶️ Launch noVNC Desktop (All-in-One)
#@markdown ---
#@markdown ## 1. Credentials (REQUIRED)
#@markdown ⚠️ **Action Required:** Add your ngrok token to Colab **Secrets (🔑)** with the name `NGROK`.
#@markdown The VNC password is fixed to: `root`
#@markdown
#@markdown ## 2. Optional: Google Drive
mount_google_drive = False #@param {type:"boolean"}
#@markdown Check this box to mount your Google Drive inside the RDP (at `/root/GoogleDrive`) for permanent file storage.
#@markdown ---

import os
import subprocess
from time import sleep
import sys
from google.colab import userdata
from google.colab import drive as colab_drive

log_file = "/content/novnc_setup_log.txt"
VNC_PASSWORD = "root"

# --- Utility Functions ---
def check_gpu():
    try:
        gpu_name = subprocess.check_output(['nvidia-smi', '--query-gpu=gpu_name', '--format=csv,noheader']).decode('utf-8').strip()
        return f"✅ GPU Detected: {gpu_name}"
    except Exception:
        return "⚠️ GPU not found. Proceeding with CPU."

def run_cmd(command, step_name=""):
    if step_name:
        print(f"  -> ⏳ {step_name}...")
    try:
        process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
        stdout, stderr = process.communicate()
        with open(log_file, 'a') as f:
            f.write(f"--- {step_name} ---\n")
            f.write(stdout)
            f.write(stderr)
        if process.returncode != 0:
            print(f"  -> ⚠️ Warning in step: {step_name}. Check log for details.")
    except Exception as e:
        print(f"  -> ❌ ERROR in step: {step_name}. {e}")
        with open(log_file, 'a') as f:
            f.write(f"--- ERROR: {step_name} ---\n{str(e)}\n")

# --- 0. System Info ---
print(f"--- 💻 SYSTEM INFO ---")
print(check_gpu())
print("-----------------------")
print(f"All installation logs will be saved to: {log_file}\n")

# --- Get NGROK Token from Secrets ---
NGROK_TOKEN = userdata.get('NGROK', None)

# --- 1. Install Dependencies ---
print("⏳ [Step 1/5] Installing Desktop & VNC servers...")
if NGROK_TOKEN is None or NGROK_TOKEN == "":
    print("❌ ERROR: ngrok token not found. Did you add it to Colab Secrets (🔑) with the name 'NGROK'?")
else:
    os.environ['DEBIAN_FRONTEND'] = 'noninteractive'
    run_cmd('apt update -y', 'Update apt')
    run_cmd('apt install -y xfce4 xfce4-goodies tigervnc-standalone-server novnc websockify python3-pyngrok', 'Install Desktop/VNC/noVNC')
    print("✅ Desktop & VNC installed.")

    # --- 2. Configure VNC ---
    print("\n⏳ [Step 2/5] Configuring VNC password...")
    os.makedirs('/root/.vnc', exist_ok=True)
    # Set VNC password
    process = subprocess.Popen(['vncpasswd', '-f'], stdin=subprocess.PIPE, stdout=subprocess.PIPE, stderr=subprocess.PIPE, text=True)
    stdout, stderr = process.communicate(input=f"{VNC_PASSWORD}\n{VNC_PASSWORD}\n")
    with open(log_file, 'a') as f:
        f.write(f"--- Set VNC password ---\n{stdout}\n{stderr}\n")
    os.chmod('/root/.vnc/passwd', 0o600)
    print("✅ VNC password set to 'root'.")

    # --- 3. Start Servers ---
    print("\n⏳ [Step 3/5] Starting VNC & noVNC servers...")
    # Start VNC server on display :1 (port 5901)
    run_cmd('vncserver :1 -localhost no -PasswordFile /root/.vnc/passwd -geometry 1920x1080 -depth 24', 'Start VNC Server')
    sleep(2)
    # Start websockify to bridge noVNC (port 6080) to VNC (port 5901)
    # Serves the noVNC web files from /usr/share/novnc/
    run_cmd('websockify -D --web /usr/share/novnc/ 6080 localhost:5901', 'Start noVNC Web Server')
    sleep(2)
    print("✅ Servers started.")

    # --- 4. Mount Google Drive (Optional) ---
    print("\n⏳ [Step 4/5] Checking for Google Drive...")
    if mount_google_drive:
        try:
            print("  -> ⏳ Mounting Google Drive...")
            colab_drive.mount('/content/drive')
            # Create a symlink into the root's home for easy access
            run_cmd('ln -s /content/drive /root/GoogleDrive', 'Create Drive shortcut')
            print("✅ Google Drive mounted to /root/GoogleDrive")
        except Exception as e:
            print(f"❌ ERROR: Could not mount drive. {e}")
    else:
        print("⚪ Skipping Google Drive mount.")

    # --- 5. Start ngrok Tunnel ---
    print("\n⏳ [Step 5/5] Starting ngrok tunnel...")
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(NGROK_TOKEN)
        # Open an HTTP tunnel to the websockify port 6080
        public_url = ngrok.connect(6080, 'http')
        print("\n\n🎉🎉🎉 \n")
        print(f"✅ noVNC Desktop is ready! Click the link below:")
        print(f"➡️  {public_url}  <-- ")
        print(f"\n(Password: {VNC_PASSWORD})")
        # Keep the cell running
        sleep(43200)
    except Exception as e:
        print(f"❌ ERROR starting ngrok: {e}")
        print("Please check your ngrok token.")
